Homework 4: Sentiment Analysis - Task 4
----

Names & Sections
----
Names: __YOUR NAMES HERE__ (Write these in every notebook you submit. For each partner, write down whether you are a 4120 or a 6120 student.)

Task 4: Neural Networks (20 points)
----

Next, we'll train a feedforward neural net to work with this data. You'll train one neural net which takes the same input as your Logistic Regression model - a sparse vector representing documents as bags of words.

In [1]:
import sentiment_utils as sutils
import numpy as np

from keras.models import Sequential
from keras.layers import Dense

# you can experiment with having some Dropout layers if you'd like to
# this is not required
from keras.layers import Dropout

# if you want to use this again
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score

import collections


C:\Users\hsubr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hsubr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# define constants for the files we are using
TRAIN_FILE = "movie_reviews_train.txt"
DEV_FILE = "movie_reviews_dev.txt"

# load in your data and make sure you understand the format
# Do not print out too much so as to impede readability of your notebook
train_tups = sutils.generate_tuples_from_file(TRAIN_FILE)
dev_tups = sutils.generate_tuples_from_file(DEV_FILE)


print(train_tups[0][0])
print(train_tups[0][1])
print(train_tups[0][2])
print(train_tups[0][3])

print(train_tups[1][:4])

# Shape of train_tups and dev_tups

#    [ [ [...], [...], [...], [...], ..... 1600 length],
#      [   0,     1,     1,     0,   ..... 1600 length ]]


# you may use either your sparse vectors or sklearn's CountVectorizer's sparse vectors
# you will experiment with multinomial and binarized representations later


['Van', 'Dien', 'must', 'cringe', 'with', 'embarrassment', 'at', 'the', 'memory', 'of', 'this', 'ludicrously', 'poor', 'film', ',', 'as', 'indeed', 'must', 'every', 'single', 'individual', 'involved', '.', 'To', 'be', 'honest', 'I', 'am', 'rather', 'embarrassed', 'to', 'admit', 'I', 'watched', 'it', 'from', 'start', 'to', 'finish', '.', 'Production', 'values', 'are', 'somewhere', 'between', 'the', 'original', 'series', 'of', "'Crossroads", "'", 'and', "'Prisoner", 'Cell', 'Block', 'H', "'", '.', 'Most', 'five', 'year', 'olds', 'would', 'be', 'able', 'to', 'come', 'up', 'with', 'more', 'realistic', 'dialogue', 'and', 'a', 'more', 'plausible', 'plot', '.', 'As', 'for', 'the', 'acting', 'performances', ',', 'if', 'you', 'can', 'imagine', 'the', 'most', 'rubbish', 'porno', 'you', 'have', 'ever', 'seen', '-', 'one', 'of', 'those', 'ones', 'where', 'the', 'action', 'is', 'padded', 'out', 'with', 'some', 'interminable', "'story", "'", 'to', 'explain', 'how', 'some', 'pouting', 'old', 'peroxid

In [3]:
#train_flat is the list of all tokens from the training tuples
train_flat = []
for row in train_tups[0]:
    train_flat.extend(row)

#test_flat is the list of all tokens from the dev tuples
test_flat = []
for row in dev_tups[0]:
    test_flat.extend(row)

#All tokens from train and dev tuples
test_flat.extend(train_flat)
vocab = set(test_flat)

reviews_train = train_tups[0]
reviews_test = dev_tups[0]

X_train_multi = sutils.featurize(vocab=vocab, data_to_be_featurized_X=reviews_train, binary=False, verbose=False)
X_test_multi = sutils.featurize(vocab=vocab, data_to_be_featurized_X=reviews_test, binary=False, verbose=False)

X_train_bi = sutils.featurize(vocab=vocab, data_to_be_featurized_X=reviews_train, binary=True, verbose=False)
X_test_bi = sutils.featurize(vocab=vocab, data_to_be_featurized_X=reviews_test, binary=True, verbose=False)

y_train = train_tups[1]
y_test = dev_tups[1]

print(f'X_train_multi: {X_train_multi}')
print(f'X_test_multi: {X_test_multi}')
print(f'X_train_bi: {X_train_bi}')
print(f'X_test_bi: {X_test_bi}')
print(f'y_train: {y_train}')
print(f'y_test: {y_test}')

X_train_multi: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
X_test_multi: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
X_train_bi: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
X_test_bi: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y_train: [0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,

In [27]:
# Create a feedforward neural network model
# that takes a sparse BoW representation of the data as input
# and makes a binary classification of positive/negative sentiment as output
# you may use any number of hidden layers >= 1 and any number of units in each hidden layer (we recommend between 50-200)
# you may use any activation function on the hidden layers 
# you should use a sigmoid activation function on the output layer
# you should use binary cross-entropy as your loss function
# sgd is an appropriate optimizer for this task
# you should report accuracy as your metric
# you may add Dropout layers if you'd like to

# create/compile your model in this cell

model_multi = Sequential()

model_multi.add(Dense(units=200, activation='relu', input_dim=X_train_multi.shape[1]))

model_multi.add(Dense(units=100, activation='tanh', input_dim=X_train_multi.shape[1]))


# put in an output layer
model_multi.add(Dense(units=1, activation='sigmoid'))


model_multi.summary()
# call compile here

model_multi.compile(loss='binary_crossentropy',
                 optimizer='sgd',
                 metrics=['accuracy'])


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 200)               6617000   
                                                                 
 dense_23 (Dense)            (None, 100)               20100     
                                                                 
 dense_24 (Dense)            (None, 1)                 101       
                                                                 
Total params: 6637201 (25.32 MB)
Trainable params: 6637201 (25.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
model_bi = Sequential()

model_bi.add(Dense(units=200, activation='relu', input_dim=X_train_multi.shape[1]))

model_bi.add(Dense(units=200, activation='tanh', input_dim=X_train_multi.shape[1]))

model_bi.add(Dense(units=200, activation='sigmoid', input_dim=X_train_multi.shape[1]))

model_bi.add(Dense(units=1, activation='sigmoid'))


model_bi.summary()

model_bi.compile(loss='binary_crossentropy',
                 optimizer='sgd',
                 metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 200)               6617000   
                                                                 
 dense_5 (Dense)             (None, 200)               40200     
                                                                 
 dense_6 (Dense)             (None, 200)               40200     
                                                                 
 dense_7 (Dense)             (None, 1)                 201       
                                                                 
Total params: 6697601 (25.55 MB)
Trainable params: 6697601 (25.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


How many trainable parameters does your model have? Our model has 6697601 trainable parameters.

In [28]:
# train your model
# Felix's computer takes about 2 sec for 3 epochs
# reports an accuracy of 0.78 at that point using the sgd optimizer

# Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})
# indicates you should change a list into a numpy array

print('Multinomial model fit')
model_multi.fit(np.array(X_train_multi), np.array(y_train), epochs=6, verbose=1)


Multinomial model fit
Epoch 1/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6764 - accuracy: 0.5869
Epoch 2/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6369 - accuracy: 0.6681
Epoch 3/6
50/50 [==============================] - 2s 31ms/step - loss: 0.5909 - accuracy: 0.6888
Epoch 4/6
50/50 [==============================] - 2s 31ms/step - loss: 0.5726 - accuracy: 0.7169
Epoch 5/6
50/50 [==============================] - 2s 31ms/step - loss: 0.5272 - accuracy: 0.7644
Epoch 6/6
50/50 [==============================] - 2s 31ms/step - loss: 0.4985 - accuracy: 0.7887


In [7]:
print('Binomial model fit')
model_bi.fit(np.array(X_train_bi), np.array(y_train), epochs=6, verbose=1)


Binomial model fit
Epoch 1/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6961 - accuracy: 0.4894
Epoch 2/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6921 - accuracy: 0.5169
Epoch 3/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6910 - accuracy: 0.5394
Epoch 4/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6897 - accuracy: 0.5337
Epoch 5/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6878 - accuracy: 0.5619
Epoch 6/6
50/50 [==============================] - 2s 31ms/step - loss: 0.6852 - accuracy: 0.5694


In [24]:
# make a prediction on the dev set
# then make a classification decision based on that prediction
# predicting all examples takes < 1 sec on Felix's computer

multi_labels = model_multi.predict(X_test_multi)

multi_assigned_labels = [1 if y_hat_val > .5 else 0 for y_hat_val in multi_labels]
print("First 5 assigned probabilities for multinomial model:", multi_labels[:5])
print("First 5 assigned labels for multinomial model:", multi_assigned_labels[:5])

print(collections.Counter(multi_assigned_labels))

7/7 [==============================] - 0s 13ms/step
First 5 assigned probabilities for multinomial model: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]
First 5 assigned labels for multinomial model: [1, 1, 1, 1, 1]
Counter({1: 200})


In [9]:
bi_labels = model_bi.predict(X_test_bi)

bi_assigned_labels = [1 if y_hat_val > .5 else 0 for y_hat_val in bi_labels]
print("First 5 assigned probabilities for binomial model:", bi_labels[:5])
print("First 5 assigned labels for binomial model:", bi_assigned_labels[:5])

7/7 [==============================] - 0s 8ms/step
First 5 assigned probabilities for binomial model: [[0.49804437]
 [0.49342486]
 [0.49592173]
 [0.5168608 ]
 [0.4983251 ]]
First 5 assigned labels for binomial model: [0, 0, 0, 1, 0]


In [10]:
# use the model.evaluate function to report the loss and accuracy on the dev set
score = model_multi.evaluate(np.array(X_test_multi), np.array(y_test), verbose=1)
print('Test multinomial model loss:', score[0])
print('Test multinomial model accuracy:', score[1])

7/7 [==============================] - 0s 9ms/step - loss: 0.6659 - accuracy: 0.5250
Test multinomial model loss: 0.6658968329429626
Test multinomial model accuracy: 0.5249999761581421


In [11]:
score = model_bi.evaluate(np.array(X_test_bi), np.array(y_test), verbose=1)
print('Test binomial model loss:', score[0])
print('Test binomial model accuracy:', score[1])

7/7 [==============================] - 0s 10ms/step - loss: 0.6867 - accuracy: 0.6750
Test binomial model loss: 0.6867132782936096
Test binomial model accuracy: 0.675000011920929


In [12]:
# create the same graph as with NB and LR, with your neural network model instead!
# make sure to re-create your model each time you train it — you don't want to start with
# an already trained network!

# For a model with one hidden layer of 50 units:
# Takes < 15 sec to run on Felix's computer w/ 3 epochs
# Takes < 30 sec to run on Felix's computer w/ 10 epochs
# Takes < 50 sec to run on Felix's computer w/ 20 epochs
# you need not train your model more than 20 epochs
# you should experiment with different numbers of epochs to see how performance varies
# you need not create an experiment that takes > 10 min to run (please do not do this)



In [13]:
print(f'F1 Score for multinomial model: {f1_score(y_test, multi_assigned_labels)}')
print(f'F1 Score for binomial model: {f1_score(y_test, bi_assigned_labels)}')

F1 Score for multinomial model: 0.6885245901639345
F1 Score for binomial model: 0.7085201793721974


Report the f1 scores for your model with the following settings, using the same number of epochs to train in both cases:
- number of epochs used: 6 epochs
- multinomial features: __YOUR ANSWER HERE__ 
- binarized features: __YOUR ANSWER HERE__